# Restoring Your SageMaker Session: Reloading Endpoints and Tuners

## Introduction

When working with long-running processes in Jupyter Notebooks, such as machine learning model training and deployment, a kernel restart can lead to the loss of all in-memory variables. However, resources created in AWS SageMaker, like endpoints and completed training jobs, persist independently of your notebook session.

This supplementary notebook provides the necessary code snippets to reconnect to your existing SageMaker endpoints and re-instantiate your hyperparameter tuner object. By doing this, you can resume your workflow, perform further evaluations, or proceed with new tasks without needing to rerun the entire original notebook.

**Prerequisite:** You must have already run the main `train_tune_deploy_register.ipynb` notebook to create the SageMaker resources (endpoints and the hyperparameter tuning job) that you intend to reload.


---

## Table of Contents

1.  [Setup and Essential Imports](#setup-supplement)
2.  [Reloading SageMaker Endpoints](#reload-endpoints)
    1.  [Define Endpoint Names](#define-endpoints)
    2.  [Create Predictor Objects](#create-predictors)
3.  [Reloading the Hyperparameter Tuner Object](#reload-tuner)
    1.  [List Completed Tuning Jobs](#list-jobs)
    2.  [Attach to the Specific Tuning Job](#attach-tuner)
    3.  [Access the Best Estimator](#access-estimator)

## 1. Setup and Essential Imports

To begin, we need to import the core libraries required to interact with SageMaker and AWS. These are the `sagemaker`, `boto3`, and specific classes like `Predictor` and `HyperparameterTuner`.


In [ ]:
import sagemaker
import boto3
from sagemaker.predictor import Predictor
from sagemaker.tuner import HyperparameterTuner
from sagemaker.serializers import CSVSerializer

# Establish a SageMaker session
sagemaker_session = sagemaker.Session()
# Initialize the boto3 client for SageMaker
sm_client = boto3.client("sagemaker")

## 2. Reloading SageMaker Endpoints

Even after a kernel restart, your deployed SageMaker endpoints are still active and ready to serve predictions. To use them, we simply need to create a `Predictor` object and point it to the existing endpoint name.

### 2.1. Define Your Endpoint Names

First, you must provide the names of the endpoints you created in your previous session.

**Action Required:** Replace the placeholder strings below with the actual names of your deployed endpoints. You can find these names in the output of the original notebook or in the SageMaker console under "Inference" -> "Endpoints".


In [ ]:
# Replace with the name of your endpoint from the initial model training
before_tuning_endpoint_name = "fraud-detection-2025-08-06-09-38-15-060"

# Replace with the name of your endpoint from the hyperparameter-tuned model
after_tuning_endpoint_name = "sagemaker-xgboost-2025-08-06-10-16-24-980"

### 2.2. Create Predictor Objects

Now, we instantiate `Predictor` objects for each endpoint. This re-establishes the connection from our notebook to the live endpoints. We also need to re-apply the `CSVSerializer`, as this setting is part of the client-side object and was lost when the kernel restarted.

In [ ]:
xgb_predictor = Predictor(
    endpoint_name=before_tuning_endpoint_name,
    sagemaker_session=sagemaker_session
)

Create a Predictor object for the tuned model's endpoint

In [ ]:
tuned_xgb_predictor = Predictor(
    endpoint_name=after_tuning_endpoint_name,
    sagemaker_session=sagemaker_session
)

Re-apply the serializer for sending data in CSV format

In [ ]:
xgb_predictor.serializer = CSVSerializer()
tuned_xgb_predictor.serializer = CSVSerializer()

In [ ]:
print(f"Successfully reconnected to non-tuned endpoint: {xgb_predictor.endpoint_name}")
print(f"Successfully reconnected to tuned endpoint: {tuned_xgb_predictor.endpoint_name}")

You can now use `xgb_predictor.predict()` and `tuned_xgb_predictor.predict()` as you did in the original notebook.

## 3. Reloading the Hyperparameter Tuner Object

The results and configuration of a completed hyperparameter tuning job are stored persistently in SageMaker. We can retrieve this information to recreate the tuner object in our notebook.


### 3.1. List Completed Tuning Jobs to Find the Name

If you don't remember the exact name of your tuning job, you can list the most recent completed jobs using the `boto3` client. This helps you identify the correct job to attach to.

In [ ]:
# List the 10 most recent completed hyperparameter tuning jobs
response = sm_client.list_hyper_parameter_tuning_jobs(
    SortBy='CreationTime',
    SortOrder='Descending',
    StatusEquals='Completed',
    MaxResults=10
)

print("Recent Completed Tuning Jobs:")
for job in response['HyperParameterTuningJobSummaries']:
    print(f"- {job['HyperParameterTuningJobName']}")

### 3.2. Attach to the Specific Tuning Job

Once you have identified the correct tuning job name from the list above, assign it to the `tuning_job_name` variable. Then, use the `HyperparameterTuner.attach()` class method to recreate the tuner object.

**Action Required:** Copy the name of your tuning job from the output above and paste it into the `tuning_job_name` variable below.

In [ ]:
# Replace with the name of your hyperparameter tuning job
tuning_job_name = "sagemaker-xgboost-250806-1008"

# Attach to the existing tuning job to recreate the tuner object
tuner = HyperparameterTuner.attach(
    tuning_job_name=tuning_job_name,
    sagemaker_session=sagemaker_session
)

print(f"Successfully attached to tuner job: {tuner.latest_tuning_job.name}")

### 3.3. Access the Best Estimator

With the `tuner` object re-instantiated, you can now access all of its properties and methods as if you had just completed the tuning job. For example, you can easily retrieve the best-performing estimator.

In [ ]:
# Retrieve the best estimator from the tuning job
best_estimator = tuner.best_estimator()

In [ ]:
print("Successfully retrieved the best estimator from the tuner.")
print(f"Best training job name: {tuner.best_training_job()}")
print(f"Model artifacts for the best estimator are at: {best_estimator.model_data}")

You can now use the `best_estimator` object to, for instance, deploy the model, register it, or analyze its hyperparameters.